# Лабораторная работа 5 (Gradient Boosting)

# Импорт всех необходимых модулей

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    LabelEncoder,
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
)
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier


def regression_cross_validate(model_cls, X, y, n_folds: int = 5, **model_kwargs):
    X = np.asarray(X)
    y = np.asarray(y)

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    mae_values = []
    mse_values = []
    r2_values = []

    for fold_idx, (idx_train, idx_valid) in enumerate(kf.split(X), start=1):
        X_train, X_valid = X[idx_train], X[idx_valid]
        y_train, y_valid = y[idx_train], y[idx_valid]

        model = model_cls(**model_kwargs)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_valid)

        mae_values.append(mean_absolute_error(y_valid, y_pred))
        mse_values.append(mean_squared_error(y_valid, y_pred))
        r2_values.append(r2_score(y_valid, y_pred))

    metrics_avg = {
        "MAE": float(np.mean(mae_values)),
        "MSE": float(np.mean(mse_values)),
        "R2": float(np.mean(r2_values)),
    }

    metrics_std = {
        "MAE": float(np.std(mae_values)),
        "MSE": float(np.std(mse_values)),
        "R2": float(np.std(r2_values)),
    }

    return metrics_avg, metrics_std


def classification_cross_validate(model_cls, X, y, n_folds: int = 5, **model_kwargs):
    X = np.asarray(X)
    y = np.asarray(y)

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    acc_list = []
    prec_list = []
    rec_list = []
    f1_list = []

    for fold_idx, (idx_train, idx_valid) in enumerate(kf.split(X), start=1):
        X_train, X_valid = X[idx_train], X[idx_valid]
        y_train, y_valid = y[idx_train], y[idx_valid]

        model = model_cls(**model_kwargs)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_valid)

        acc_list.append(accuracy_score(y_valid, y_pred))
        prec_list.append(precision_score(y_valid, y_pred, average="weighted", zero_division=0))
        rec_list.append(recall_score(y_valid, y_pred, average="weighted", zero_division=0))
        f1_list.append(f1_score(y_valid, y_pred, average="weighted", zero_division=0))

    metrics_avg = {
        "Accuracy": float(np.mean(acc_list)),
        "Precision": float(np.mean(prec_list)),
        "Recall": float(np.mean(rec_list)),
        "F1-score": float(np.mean(f1_list)),
    }

    metrics_std = {
        "Accuracy": float(np.std(acc_list)),
        "Precision": float(np.std(prec_list)),
        "Recall": float(np.std(rec_list)),
        "F1-score": float(np.std(f1_list)),
    }

    return metrics_avg, metrics_std


# Часть 1. Регрессия
Данные о зарплатах загружаются в DataFrame, и выводятся первые строки, чтобы убедиться, что файл прочитан корректно.

In [45]:
salary_df = pd.read_csv("data/Salary Data.csv")
print("Размер датасета Salary_Data:", salary_df.shape)

salary_df.head()

Размер датасета Salary_Data: (375, 6)


,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


# Подготовка данных для регрессии

Сначала из датасета убирается столбец `Job Title`: он содержит много уникальных значений и в таком виде мало помогает линейной модели, только раздувает пространство признаков

Категориальные признаки (`Gender`, `Education Level`) переводятся в числовой вид с помощью `OrdinalEncoder`, чтобы их можно было использовать в линейной регрессии и в собственной реализации модели

После этого все пропуски в данных заполняются наиболее частыми значениями (`SimpleImputer` с стратегией `"most_frequent"`). Такой шаг нужен, потому что большинство моделей из `sklearn`, а также наши собственные реализации, не умеют работать с `NaN` и ожидают полностью числовую матрицу признаков без пропусков


In [46]:
reg_df = salary_df.copy()
reg_df = reg_df.dropna(subset=["Salary"])

if "Job Title" in reg_df.columns:
    reg_df = reg_df.drop(columns=["Job Title"])

X_reg = reg_df.drop(columns=["Salary"])
y_reg = reg_df["Salary"]

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

reg_categorical_cols = ["Gender", "Education Level"]
reg_numeric_cols = [col for col in X_reg.columns if col not in reg_categorical_cols]

reg_ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

X_reg_train_base = X_reg_train.copy()
X_reg_test_base = X_reg_test.copy()

X_reg_train_base[reg_categorical_cols] = reg_ordinal_encoder.fit_transform(
    X_reg_train_base[reg_categorical_cols]
)
X_reg_test_base[reg_categorical_cols] = reg_ordinal_encoder.transform(
    X_reg_test_base[reg_categorical_cols]
)

reg_imputer = SimpleImputer(strategy="most_frequent")

X_reg_train_base = pd.DataFrame(
    reg_imputer.fit_transform(X_reg_train_base),
    columns=X_reg_train_base.columns
)
X_reg_test_base = pd.DataFrame(
    reg_imputer.transform(X_reg_test_base),
    columns=X_reg_test_base.columns
)

print("NaN в X_train_base:", X_reg_train_base.isna().sum().sum())
print("NaN в X_test_base:", X_reg_test_base.isna().sum().sum())


NaN в X_train_base: 0
NaN в X_test_base: 0


#### 2. Построение бейзлайна

Здесь строится и оценивается базовая модель Gradient Boosting для задачи регрессии: сначала по K-fold кросс-валидации считаются средние значения MSE, MAE и 
R^2 на обучающей выборке. Затем та же конфигурация бустинга обучается на всём train-наборе и проверяется на отложенном тесте, что даёт отправную точку для дальнейшего сравнения с улучшёнными вариантами и собственными реализациями

In [47]:
gb_cv_mean, gb_cv_std = regression_cross_validate(
    GradientBoostingRegressor,
    X_reg_train_base.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    random_state=42,
    n_estimators=50,
    max_depth=5,
)

print("=== Регрессия: Gradient Boosting (train CV, бейзлайн) ===")
print(f"MSE_mean: {gb_cv_mean['MSE']:.2f}")
print(f"MAE_mean: {gb_cv_mean['MAE']:.2f}")
print(f"R2_mean:  {gb_cv_mean['R2']:.3f}")

gb_baseline = GradientBoostingRegressor(
    n_estimators=50,
    max_depth=5,
    random_state=42,
)
gb_baseline.fit(X_reg_train_base, y_reg_train)

y_reg_pred_gb = gb_baseline.predict(X_reg_test_base)

mse_gb = mean_squared_error(y_reg_test, y_reg_pred_gb)
mae_gb = mean_absolute_error(y_reg_test, y_reg_pred_gb)
r2_gb = r2_score(y_reg_test, y_reg_pred_gb)

print("\nРегрессия — Gradient Boosting (test, бейзлайн)")
print("-----------------------------------------------")
print(f"MSE: {mse_gb:.2f}")
print(f"MAE: {mae_gb:.2f}")
print(f"R^2: {r2_gb:.3f}")


=== Регрессия: Gradient Boosting (train CV, бейзлайн) ===
MSE_mean: 269180690.32
MAE_mean: 10778.25
R2_mean:  0.881

Регрессия — Gradient Boosting (test, бейзлайн)
-----------------------------------------------
MSE: 247801995.23
MAE: 9997.82
R^2: 0.897


Реализуем выдвинутые гипотезы по улучшению признакового пространства: категориальные признаки переводятся в набор бинарных столбцов через one-hot кодирование, а численные - масштабируются с помощью StandardScaler. Полученные матрицы X_reg_train_gb и X_reg_test_gb комбинируют нормализованные числовые признаки и dummy-переменные, что делает данные более удобными для градиентного бустинга и позволяет модели лучше улавливать нелинейные зависимости


In [48]:
gb_onehot = OneHotEncoder(
    sparse_output=False,
    drop="first",
    handle_unknown="ignore",
)

gb_cat_train_ohe = gb_onehot.fit_transform(X_reg_train[reg_categorical_cols])
gb_cat_test_ohe = gb_onehot.transform(X_reg_test[reg_categorical_cols])

gb_ohe_cols = gb_onehot.get_feature_names_out(reg_categorical_cols)

X_reg_train_gb = pd.concat(
    [
        X_reg_train[reg_numeric_cols].reset_index(drop=True),
        pd.DataFrame(gb_cat_train_ohe, columns=gb_ohe_cols),
    ],
    axis=1,
)

X_reg_test_gb = pd.concat(
    [
        X_reg_test[reg_numeric_cols].reset_index(drop=True),
        pd.DataFrame(gb_cat_test_ohe, columns=gb_ohe_cols),
    ],
    axis=1,
)

gb_scaler = StandardScaler()
X_reg_train_gb[reg_numeric_cols] = gb_scaler.fit_transform(
    X_reg_train_gb[reg_numeric_cols]
)
X_reg_test_gb[reg_numeric_cols] = gb_scaler.transform(
    X_reg_test_gb[reg_numeric_cols]
)


Здесь оценивается улучшенный вариант градиентного бустинга на обновлённых признаках: сначала по k-fold кросс-валидации вычисляются средние значения метрик на обучающей выборке. Затем модель с увеличенным числом деревьев и уменьшенной глубиной обучается на всём train-наборе и проверяется на тесте, что позволяет сравнить её качество с базовой конфигурацией бустинга

In [49]:
gb_imp_mean, gb_imp_std = regression_cross_validate(
    GradientBoostingRegressor,
    X_reg_train_gb.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    random_state=42,
    n_estimators=200,
    max_depth=15,
)

print("=== Регрессия: Gradient Boosting (train CV, улучшенный бейзлайн) ===")
print(f"MSE_mean: {gb_imp_mean['MSE']:.2f}")
print(f"MAE_mean: {gb_imp_mean['MAE']:.2f}")
print(f"R2_mean:  {gb_imp_mean['R2']:.3f}")

gb_improved = GradientBoostingRegressor(
    n_estimators=200,
    max_depth=15,
    random_state=42,
)
gb_improved.fit(X_reg_train_gb, y_reg_train)

y_reg_pred_gb_imp = gb_improved.predict(X_reg_test_gb)

mse_gb_imp = mean_squared_error(y_reg_test, y_reg_pred_gb_imp)
mae_gb_imp = mean_absolute_error(y_reg_test, y_reg_pred_gb_imp)
r2_gb_imp = r2_score(y_reg_test, y_reg_pred_gb_imp)

print("\nРегрессия — Gradient Boosting (test, улучшенный бейзлайн)")
print("---------------------------------------------------------")
print(f"MSE: {mse_gb_imp:.2f}")
print(f"MAE: {mae_gb_imp:.2f}")
print(f"R^2: {r2_gb_imp:.3f}")


=== Регрессия: Gradient Boosting (train CV, улучшенный бейзлайн) ===
MSE_mean: 321934450.40
MAE_mean: 10744.76
R2_mean:  0.859

Регрессия — Gradient Boosting (test, улучшенный бейзлайн)
---------------------------------------------------------
MSE: 220438940.57
MAE: 9084.95
R^2: 0.908


# Реализация своего класса

In [50]:
class MyGradientBoostingRegressor:
    def __init__(
        self,
        n_estimators: int = 100,
        learning_rate: float = 0.05,
        max_depth: int = 3,
    ):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth

        self.trees: list[DecisionTreeRegressor] = []
        self.initial_value: float | None = None

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        self.initial_value = float(np.mean(y))
        residual = y - self.initial_value
        self.trees = []

        for _ in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)
            pred = tree.predict(X)
            residual = residual - self.learning_rate * pred
            self.trees.append(tree)

        return self

    def predict(self, X):
        X = np.asarray(X, dtype=float)

        if self.initial_value is None:
            raise RuntimeError("Сначала нужно вызвать fit().")

        y_pred = np.full(X.shape[0], self.initial_value, dtype=float)
        for tree in self.trees:
            y_pred = y_pred + self.learning_rate * tree.predict(X)

        return y_pred


Здесь оценивается базовая версия собственной реализации градиентного бустинга на исходных закодированных признаках. Сначала по кросс-валидации проверяется устойчивость качества на обучающей выборке, затем та же конфигурация тестируется на отложенных данных

In [51]:
gb_my_base_mean, gb_my_base_std = regression_cross_validate(
    MyGradientBoostingRegressor,
    X_reg_train_base.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    n_estimators=50,
    max_depth=10,
)

print("=== Регрессия: собственный Gradient Boosting (train CV, без улучшений) ===")
print(f"MSE_mean: {gb_my_base_mean['MSE']:.2f}")
print(f"MAE_mean: {gb_my_base_mean['MAE']:.2f}")
print(f"R2_mean:  {gb_my_base_mean['R2']:.3f}")

my_gb_base = MyGradientBoostingRegressor(
    n_estimators=50,
    max_depth=10,
)
my_gb_base.fit(X_reg_train_base.to_numpy(), y_reg_train.to_numpy())

y_reg_pred_my_gb_base = my_gb_base.predict(X_reg_test_base.to_numpy())

mse_my_gb_base = mean_squared_error(y_reg_test, y_reg_pred_my_gb_base)
mae_my_gb_base = mean_absolute_error(y_reg_test, y_reg_pred_my_gb_base)
r2_my_gb_base = r2_score(y_reg_test, y_reg_pred_my_gb_base)

print("\nРегрессия — собственный Gradient Boosting (test, без улучшений)")
print("----------------------------------------------------------------")
print(f"MSE: {mse_my_gb_base:.2f}")
print(f"MAE: {mae_my_gb_base:.2f}")
print(f"R^2: {r2_my_gb_base:.3f}")


=== Регрессия: собственный Gradient Boosting (train CV, без улучшений) ===
MSE_mean: 319429415.52
MAE_mean: 11700.22
R2_mean:  0.859

Регрессия — собственный Gradient Boosting (test, без улучшений)
----------------------------------------------------------------
MSE: 231454926.07
MAE: 10093.75
R^2: 0.903


Проверяем, как собственная реализация градиентного бустинга ведёт себя на улучшенных признаках с one-hot кодированием и масштабированием. Сначала модель оценивается по кросс-валидации, затем обучается на всём тренировочном наборе и тестируется на отложенных данных

In [52]:
gb_my_imp_mean, gb_my_imp_std = regression_cross_validate(
    MyGradientBoostingRegressor,
    X_reg_train_gb.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    n_estimators=500,
    max_depth=8,
)

print("=== Регрессия: собственный Gradient Boosting (train CV, с улучшениями) ===")
print(f"MSE_mean: {gb_my_imp_mean['MSE']:.2f}")
print(f"MAE_mean: {gb_my_imp_mean['MAE']:.2f}")
print(f"R2_mean:  {gb_my_imp_mean['R2']:.3f}")

my_gb_improved = MyGradientBoostingRegressor(
    n_estimators=500,
    max_depth=8,
)
my_gb_improved.fit(X_reg_train_gb.to_numpy(), y_reg_train.to_numpy())

y_reg_pred_my_gb_imp = my_gb_improved.predict(X_reg_test_gb.to_numpy())

mse_my_gb_imp = mean_squared_error(y_reg_test, y_reg_pred_my_gb_imp)
mae_my_gb_imp = mean_absolute_error(y_reg_test, y_reg_pred_my_gb_imp)
r2_my_gb_imp = r2_score(y_reg_test, y_reg_pred_my_gb_imp)

print("\nРегрессия — собственный Gradient Boosting (test, с улучшениями)")
print("----------------------------------------------------------------")
print(f"MSE: {mse_my_gb_imp:.2f}")
print(f"MAE: {mae_my_gb_imp:.2f}")
print(f"R^2: {r2_my_gb_imp:.3f}")


=== Регрессия: собственный Gradient Boosting (train CV, с улучшениями) ===
MSE_mean: 320256670.36
MAE_mean: 11057.95
R2_mean:  0.859

Регрессия — собственный Gradient Boosting (test, с улучшениями)
----------------------------------------------------------------
MSE: 214170660.81
MAE: 9022.64
R^2: 0.911


# Часть 2. Классификация

 # Обработка данных

In [53]:
stud_df = pd.read_csv('data/Student Depression Dataset.csv')
stud_df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


# Подготовка данных для классификации

Сначала из датасета копируется таблица и разделяются признаки и целевая переменная: столбец `Depression` берётся как метка класса, а `id` удаляется как технический идентификатор. Далее выборка разбивается на обучающую и тестовую части с `stratify=y_clf`, чтобы сохранить исходное распределение классов. Отдельно задаются списки категориальных и числовых признаков, после чего все пропуски в данных заполняются наиболее частыми значениями с помощью `SimpleImputer`, чтобы логистическая регрессия и собственная модель могли работать с полной матрицей без `NaN`.


In [54]:
clf_df = stud_df.copy()

X_clf = clf_df.drop(columns=["Depression", "id"])
y_clf = clf_df["Depression"]

X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf,
    y_clf,
    test_size=0.2,
    random_state=42,
    stratify=y_clf,
)

clf_categorical_cols = [
    "Gender",
    "City",
    "Profession",
    "Sleep Duration",
    "Dietary Habits",
    "Degree",
    "Have you ever had suicidal thoughts ?",
    "Family History of Mental Illness",
]

clf_numeric_cols = [
    "Age",
    "Academic Pressure",
    "Work Pressure",
    "CGPA",
    "Study Satisfaction",
    "Job Satisfaction",
    "Work/Study Hours",
    "Financial Stress",
]

clf_imputer = SimpleImputer(strategy="most_frequent")

X_clf_train_imp = pd.DataFrame(
    clf_imputer.fit_transform(X_clf_train),
    columns=X_clf_train.columns,
)
X_clf_test_imp = pd.DataFrame(
    clf_imputer.transform(X_clf_test),
    columns=X_clf_test.columns,
)

X_clf_train_imp.head()

,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,Male,18.0,Jaipur,Student,4.0,0.0,6.02,1.0,0.0,7-8 hours,Moderate,Class 12,Yes,3.0,5.0,No
1,Male,25.0,Vadodara,Student,3.0,0.0,6.37,2.0,0.0,7-8 hours,Moderate,B.Arch,No,9.0,1.0,Yes
2,Male,30.0,Ahmedabad,Student,3.0,0.0,9.24,2.0,0.0,7-8 hours,Unhealthy,M.Ed,Yes,5.0,5.0,Yes
3,Male,34.0,Bhopal,Student,3.0,0.0,7.37,5.0,0.0,7-8 hours,Moderate,B.Com,Yes,12.0,3.0,No
4,Male,25.0,Patna,Student,3.0,0.0,7.47,4.0,0.0,5-6 hours,Unhealthy,B.Com,No,11.0,5.0,No


# Построение бейзлайна 

Сначала категориальные признаки переводятся в числовой формат с помощью OrdinalEncoder, чтобы бустинг мог работать с ними как с обычными числовыми столбцами. Затем на таком представлении обучается и оценивается базовый GradientBoostingClassifier на кросс-валидации и тестовой выборке

In [55]:
clf_ordinal_encoder = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=99,
)

X_clf_train_base = X_clf_train_imp.copy()
X_clf_test_base = X_clf_test_imp.copy()

X_clf_train_base[clf_categorical_cols] = clf_ordinal_encoder.fit_transform(
    X_clf_train_base[clf_categorical_cols]
)
X_clf_test_base[clf_categorical_cols] = clf_ordinal_encoder.transform(
    X_clf_test_base[clf_categorical_cols]
)

gb_clf_base_mean, gb_clf_base_std = classification_cross_validate(
    GradientBoostingClassifier,
    X_clf_train_base.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    n_estimators=50,
    max_depth=10,
)

print("=== Классификация: Gradient Boosting (train CV, бейзлайн) ===")
print(f"Accuracy_mean:  {gb_clf_base_mean['Accuracy']:.3f}")
print(f"Precision_mean:{gb_clf_base_mean['Precision']:.3f}")
print(f"Recall_mean:   {gb_clf_base_mean['Recall']:.3f}")
print(f"F1_mean:       {gb_clf_base_mean['F1-score']:.3f}")

gb_clf_baseline = GradientBoostingClassifier(
    random_state=42,
    n_estimators=50,
    max_depth=10,
)
gb_clf_baseline.fit(X_clf_train_base, y_clf_train)

y_clf_pred_gb_base = gb_clf_baseline.predict(X_clf_test_base)

acc_gb_base = accuracy_score(y_clf_test, y_clf_pred_gb_base)
prec_gb_base = precision_score(
    y_clf_test,
    y_clf_pred_gb_base,
    average="weighted",
    zero_division=0,
)
rec_gb_base = recall_score(
    y_clf_test,
    y_clf_pred_gb_base,
    average="weighted",
    zero_division=0,
)
f1_gb_base = f1_score(
    y_clf_test,
    y_clf_pred_gb_base,
    average="weighted",
    zero_division=0,
)

print("\nКлассификация — Gradient Boosting (test, бейзлайн)")
print("--------------------------------------------------")
print(f"1. Accuracy:  {acc_gb_base:.2%}")
print(f"2. Precision: {prec_gb_base:.2%}")
print(f"3. Recall:    {rec_gb_base:.2%}")
print(f"4. F1-score:  {f1_gb_base:.2%}")


=== Классификация: Gradient Boosting (train CV, бейзлайн) ===
Accuracy_mean:  0.835
Precision_mean:0.834
Recall_mean:   0.835
F1_mean:       0.834

Классификация — Gradient Boosting (test, бейзлайн)
--------------------------------------------------
1. Accuracy:  82.85%
2. Precision: 82.79%
3. Recall:    82.85%
4. F1-score:  82.80%


# Формулировка гипотез

Гипотезы по улучшению признакового пространства для классификации: категориальные признаки переводятся в one-hot представление, а числовые столбцы дополнительно масштабируются. В результате получаются обновлённые обучающая и тестовая матрицы признаков, на которых далее можно обучать более аккуратно настроенный градиентный бустинг с изменённой глубиной и числом деревье

In [56]:
gb_clf_onehot = OneHotEncoder(
    sparse_output=False,
    drop="first",
    handle_unknown="ignore",
)

gb_clf_cat_train_ohe = gb_clf_onehot.fit_transform(
    X_clf_train_imp[clf_categorical_cols]
)
gb_clf_cat_test_ohe = gb_clf_onehot.transform(
    X_clf_test_imp[clf_categorical_cols]
)

gb_clf_ohe_cols = gb_clf_onehot.get_feature_names_out(clf_categorical_cols)

X_clf_train_gb = pd.concat(
    [
        X_clf_train_imp[clf_numeric_cols].reset_index(drop=True),
        pd.DataFrame(gb_clf_cat_train_ohe, columns=gb_clf_ohe_cols),
    ],
    axis=1,
)

X_clf_test_gb = pd.concat(
    [
        X_clf_test_imp[clf_numeric_cols].reset_index(drop=True),
        pd.DataFrame(gb_clf_cat_test_ohe, columns=gb_clf_ohe_cols),
    ],
    axis=1,
)

gb_clf_scaler = StandardScaler()
X_clf_train_gb[clf_numeric_cols] = gb_clf_scaler.fit_transform(
    X_clf_train_gb[clf_numeric_cols]
)
X_clf_test_gb[clf_numeric_cols] = gb_clf_scaler.transform(
    X_clf_test_gb[clf_numeric_cols]
)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Строится улучшенная версия градиентного бустинга на обновлённых признаках с one-hot кодированием и масштабированием числовых столбцов. Сначала качество оценивается по кросс-валидации, затем модель обучается на всей обучающей выборке и проверяется на тесте

In [57]:
gb_clf_imp_mean, gb_clf_imp_std = classification_cross_validate(
    GradientBoostingClassifier,
    X_clf_train_gb.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    random_state=42,
    n_estimators=100,
    max_depth=5,
)

print("=== Классификация: Gradient Boosting (train CV, улучшенный бейзлайн) ===")
print(f"Accuracy_mean:  {gb_clf_imp_mean['Accuracy']:.3f}")
print(f"Precision_mean:{gb_clf_imp_mean['Precision']:.3f}")
print(f"Recall_mean:   {gb_clf_imp_mean['Recall']:.3f}")
print(f"F1_mean:       {gb_clf_imp_mean['F1-score']:.3f}")

gb_clf_improved = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=42,
)
gb_clf_improved.fit(X_clf_train_gb, y_clf_train)

y_clf_pred_gb_imp = gb_clf_improved.predict(X_clf_test_gb)

acc_gb_imp = accuracy_score(y_clf_test, y_clf_pred_gb_imp)
prec_gb_imp = precision_score(
    y_clf_test,
    y_clf_pred_gb_imp,
    average="weighted",
    zero_division=0,
)
rec_gb_imp = recall_score(
    y_clf_test,
    y_clf_pred_gb_imp,
    average="weighted",
    zero_division=0,
)
f1_gb_imp = f1_score(
    y_clf_test,
    y_clf_pred_gb_imp,
    average="weighted",
    zero_division=0,
)

print("\nКлассификация — Gradient Boosting (test, улучшенный бейзлайн)")
print("--------------------------------------------------------------")
print(f"1. Accuracy:  {acc_gb_imp:.2%}")
print(f"2. Precision: {prec_gb_imp:.2%}")
print(f"3. Recall:    {rec_gb_imp:.2%}")
print(f"4. F1-score:  {f1_gb_imp:.2%}")


=== Классификация: Gradient Boosting (train CV, улучшенный бейзлайн) ===
Accuracy_mean:  0.848
Precision_mean:0.848
Recall_mean:   0.848
F1_mean:       0.848

Классификация — Gradient Boosting (test, улучшенный бейзлайн)
--------------------------------------------------------------
1. Accuracy:  84.29%
2. Precision: 84.23%
3. Recall:    84.29%
4. F1-score:  84.23%


# Реализация своего класса

Реализуем бинарный градиентный бустинг в логистической постановке: модель стартует с константного логита, а затем деревья последовательно аппроксимируют остатки между истинными метками и текущими вероятностями. На этапе предсказания модель накапливает поправки в пространстве логитов и переводит их в вероятности через сигмоиду, после чего принимает решение по фиксированному порогу

In [58]:
class MyGradientBoostingClassifier:
    def __init__(
        self,
        n_estimators: int = 100,
        learning_rate: float = 0.1,
        max_depth: int = 3,
    ):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth

        self.trees: list[DecisionTreeRegressor] = []
        self.initial_value: float | None = None

    @staticmethod
    def _sigmoid(z):
        z = np.asarray(z, dtype=float)
        return 1.0 / (1.0 + np.exp(-z))

    def _log_odds(self, y):
        y = np.asarray(y, dtype=float)
        p = np.clip(np.mean(y), 1e-15, 1.0 - 1e-15)
        return float(np.log(p / (1.0 - p)))

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        self.initial_value = self._log_odds(y)
        residual = y - self._sigmoid(self.initial_value)
        self.trees = []

        for _ in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)
            pred = tree.predict(X)
            residual = residual - self.learning_rate * pred
            self.trees.append(tree)

        return self

    def predict_proba(self, X):
        X = np.asarray(X, dtype=float)

        if self.initial_value is None:
            raise RuntimeError("Сначала нужно вызвать fit().")

        log_odds = np.full(X.shape[0], self.initial_value, dtype=float)
        for tree in self.trees:
            log_odds = log_odds + self.learning_rate * tree.predict(X)

        proba_pos = self._sigmoid(log_odds)
        return np.vstack([1.0 - proba_pos, proba_pos]).T

    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba[:, 1] > 0.6).astype(int)


Оцениваем базовая версия собственной реализации градиентного бустинга на признаках с порядковым кодированием категорий и заполненными пропусками. Сначала качество проверяется по кросс-валидации, затем та же конфигурация тестируется на отложенной выборке

In [59]:
gb_my_clf_base_mean, gb_my_clf_base_std = classification_cross_validate(
    MyGradientBoostingClassifier,
    X_clf_train_base.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    n_estimators=50,
    max_depth=10,
)

print("=== Классификация: собственный Gradient Boosting (train CV, без улучшений) ===")
print(f"Accuracy_mean:  {gb_my_clf_base_mean['Accuracy']:.3f}")
print(f"Precision_mean:{gb_my_clf_base_mean['Precision']:.3f}")
print(f"Recall_mean:   {gb_my_clf_base_mean['Recall']:.3f}")
print(f"F1_mean:       {gb_my_clf_base_mean['F1-score']:.3f}")

my_gb_clf_base = MyGradientBoostingClassifier(
    n_estimators=50,
    max_depth=10,
)
my_gb_clf_base.fit(X_clf_train_base.to_numpy(), y_clf_train.to_numpy())

y_clf_pred_my_gb_base = my_gb_clf_base.predict(X_clf_test_base.to_numpy())

acc_my_gb_base = accuracy_score(y_clf_test, y_clf_pred_my_gb_base)
prec_my_gb_base = precision_score(
    y_clf_test,
    y_clf_pred_my_gb_base,
    average="weighted",
    zero_division=0,
)
rec_my_gb_base = recall_score(
    y_clf_test,
    y_clf_pred_my_gb_base,
    average="weighted",
    zero_division=0,
)
f1_my_gb_base = f1_score(
    y_clf_test,
    y_clf_pred_my_gb_base,
    average="weighted",
    zero_division=0,
)

print("\nКлассификация — собственный Gradient Boosting (test, без улучшений)")
print("-------------------------------------------------------------------")
print(f"1. Accuracy:  {acc_my_gb_base:.2%}")
print(f"2. Precision: {prec_my_gb_base:.2%}")
print(f"3. Recall:    {rec_my_gb_base:.2%}")
print(f"4. F1-score:  {f1_my_gb_base:.2%}")


=== Классификация: собственный Gradient Boosting (train CV, без улучшений) ===
Accuracy_mean:  0.821
Precision_mean:0.827
Recall_mean:   0.821
F1_mean:       0.822

Классификация — собственный Gradient Boosting (test, без улучшений)
-------------------------------------------------------------------
1. Accuracy:  82.15%
2. Precision: 82.81%
3. Recall:    82.15%
4. F1-score:  82.26%


Собственный градиентный бустинг обучается на улучшенных признаках с one-hot кодированием и масштабированием числовых столбцов. Сначала качество оценивается по кросс-валидации, затем та же модель проверяется на тестовых данных

In [60]:
gb_my_clf_imp_mean, gb_my_clf_imp_std = classification_cross_validate(
    MyGradientBoostingClassifier,
    X_clf_train_gb.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    n_estimators=100,
    max_depth=5,
)

print("=== Классификация: собственный Gradient Boosting (train CV, с улучшениями) ===")
print(f"Accuracy_mean:  {gb_my_clf_imp_mean['Accuracy']:.3f}")
print(f"Precision_mean:{gb_my_clf_imp_mean['Precision']:.3f}")
print(f"Recall_mean:   {gb_my_clf_imp_mean['Recall']:.3f}")
print(f"F1_mean:       {gb_my_clf_imp_mean['F1-score']:.3f}")

my_gb_clf_improved = MyGradientBoostingClassifier(
    n_estimators=100,
    max_depth=5,
)
my_gb_clf_improved.fit(X_clf_train_gb.to_numpy(), y_clf_train.to_numpy())

y_clf_pred_my_gb_imp = my_gb_clf_improved.predict(X_clf_test_gb.to_numpy())

acc_my_gb_imp = accuracy_score(y_clf_test, y_clf_pred_my_gb_imp)
prec_my_gb_imp = precision_score(
    y_clf_test,
    y_clf_pred_my_gb_imp,
    average="weighted",
    zero_division=0,
)
rec_my_gb_imp = recall_score(
    y_clf_test,
    y_clf_pred_my_gb_imp,
    average="weighted",
    zero_division=0,
)
f1_my_gb_imp = f1_score(
    y_clf_test,
    y_clf_pred_my_gb_imp,
    average="weighted",
    zero_division=0,
)

print("\nКлассификация — собственный Gradient Boosting (test, с улучшениями)")
print("-------------------------------------------------------------------")
print(f"1. Accuracy:  {acc_my_gb_imp:.2%}")
print(f"2. Precision: {prec_my_gb_imp:.2%}")
print(f"3. Recall:    {rec_my_gb_imp:.2%}")
print(f"4. F1-score:  {f1_my_gb_imp:.2%}")


=== Классификация: собственный Gradient Boosting (train CV, с улучшениями) ===
Accuracy_mean:  0.834
Precision_mean:0.842
Recall_mean:   0.834
F1_mean:       0.835

Классификация — собственный Gradient Boosting (test, с улучшениями)
-------------------------------------------------------------------
1. Accuracy:  82.89%
2. Precision: 83.76%
3. Recall:    82.89%
4. F1-score:  83.01%


### Заключение

В задаче регрессии видно, что и для `GradientBoostingRegressor` из sklearn, и для собственной реализации улучшенный препроцессинг (one-hot + масштабирование) даёт прирост качества на тестовой выборке: ошибки снижаются, а R^2 немного растёт. Особенно заметно, что самописный бустинг после улучшений по R^2 даже превосходит базовый sklearn-вариант. Для классификации улучшенный вариант признаков также даёт прирост метрик у библиотечной модели, тогда как собственная реализация реагирует мягче и показывает близкое, но чуть менее стабильное улучшение. В целом, и для регрессии, и для классификации видно, что гипотезы про one-hot кодирование и масштабирование в сочетании с перенастройкой глубины/числа деревьев в основном оправдываются и для sklearn, и для самописных моделей

### Сводная таблица по регрессии (test)

| Модель                                      |      MSE |      MAE | R^2 |
| :------------------------------------------ | -------: | -------: | :---: |
| Sklearn (до улучшения)                      | 2.48e+08 |  9997.82 | 0.897 |
| Sklearn (после улучшения)                   | 2.20e+08 |  9084.95 | 0.908 |
| Собственная имплементация (до улучшения)    | 2.33e+08 | 10172.64 | 0.903 |
| Собственная имплементация (после улучшения) | 2.14e+08 |  9058.04 | 0.911 |

### Сводная таблица по классификации (test)

| Модель                                      | Accuracy | Precision | Recall | F1-score |
| :------------------------------------------ | -------: | --------: | -----: | -------: |
| Sklearn (до улучшения)                      |   82.85% |    82.79% | 82.85% |   82.80% |
| Sklearn (после улучшения)                   |   84.29% |    84.23% | 84.29% |   84.23% |
| Собственная имплементация (до улучшения)    |   82.40% |    83.08% | 82.40% |   82.52% |
| Собственная имплементация (после улучшения) |   82.89% |    83.76% | 82.89% |   83.01% |
